# EDA 5: Carga II - Clases y funciones BBDD e inserción

In [42]:
#importamos las librerías:

import pandas as pd
import mysql.connector

### Creamos los dos dataframes con los que tendremos que trabajar.

In [43]:
df_at = pd.read_csv('datos/ETL-att-minus.csv', index_col=0)
df_at.head()

,year,type,country,activity,age,species,month,fatal,sex
0,2018,Boating,usa,Paddling,57.0,White Shark,Jun,N,F
2,2018,Unprovoked,usa,Walking,15.0,Bull Shark,May,N,M
3,2018,Provoked,australia,Feeding sharks,32.0,Grey Shark,May,N,M
6,2018,Unprovoked,australia,Surfing,60.0,Unspecified,Apr,N,M
8,2018,Unprovoked,south africa,Paddle-skiing,33.0,White Shark,Apr,N,M


In [44]:
df_cl = pd.read_csv('datos/clima-clase.csv', index_col=0)
df_cl.head()

,wind_speed_950mb,wind_speed_900mb,wind_speed_850mb,wind_speed_800mb,wind_speed_750mb,wind_speed_700mb,wind_speed_650mb,wind_speed_600mb,wind_speed_550mb,wind_speed_500mb,...,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,2,2,3,4,4,5,5,5,5,5,...,13,15,5,1024,none,0,0,265,2,usa
1,2,3,4,5,5,5,5,5,5,6,...,12,15,5,1024,none,0,0,345,2,usa
2,2,2,2,3,3,5,5,5,5,6,...,13,15,4,1024,none,0,0,260,2,usa
3,2,3,3,3,3,4,5,6,6,6,...,15,10,2,1024,none,0,0,175,2,usa
4,2,3,3,3,4,5,6,6,6,6,...,17,6,3,1024,none,0,0,350,2,usa


### Creamos la clase según las funciones que generamos en nuestro último pair (Carga I).

In [45]:
class Carga:

    def __init__(self, nombre_bbdd):

        self.nombre_bbdd = nombre_bbdd
    
    # definimos la primera función para crear la bbdd:
    def make_ddbb(self,nombre_bbdd):
        
        self.nombre_bbdd = nombre_bbdd
  
        cnx = mysql.connector.connect(host='127.0.0.1', user='root',
        password='AlumnaAdalab')

        mc = cnx.cursor()

        try:
          mc.execute(f"CREATE DATABASE IF NOT EXISTS {nombre_bbdd};")
          print(mc)
    # en caso de que podamos ejecutar la query devuelvenos un error para saber en que nos estamos equivocando
        except mysql.connector.Error as err:
          print(err)
          print("Error Code:", err.errno)
          print("SQLSTATE", err.sqlstate)
          print("Message", err.msg)

        return nombre_bbdd
    
    #generamos la función para ejecutar una query
    def create_insert_table(self, nombre_bbdd, query):

        self.query = query
        self.nombre_bbdd = nombre_bbdd
    
        cnx = mysql.connector.connect(user='root', password=f"AlumnaAdalab",
                                     host='127.0.0.1', database=f"{nombre_bbdd}")
    
        mycursor = cnx.cursor()
 
        try: 
            mycursor.execute(query)
            cnx.commit() 
    # en caso de que podamos ejecutar la query devuelvenos un error para saber en que nos estamos equivocando
        except mysql.connector.Error as err:
          print(err)
          print("Error Code:", err.errno)
          print("SQLSTATE", err.sqlstate)
          print("Message", err.msg)

### Aplicación de la clase:

- 1. Definimos la clase en una variable.

In [46]:
carga = Carga('tiburones-2')

- 2. Llamamos a la primera función para crear la bbdd.

In [47]:
# Le hemos pedido a la función que el return sea el nombre de la bbdd.
bbdd_a = carga.make_ddbb('tiburones2')

MySQLCursor: CREATE DATABASE IF NOT EXISTS tiburones2..


- 3. Generamos una query para crear una tabla.

In [48]:
attacks = ''' 
    CREATE TABLE IF NOT EXISTS `attacks` (
    `year` INT DEFAULT NULL,
    `type` VARCHAR(50) DEFAULT NULL, 
    `country` VARCHAR(50) DEFAULT NULL,  
    `activity` VARCHAR(100) DEFAULT NULL,  
    `age` FLOAT DEFAULT NULL,
    `species` VARCHAR(50) DEFAULT NULL,
    `month` VARCHAR (45) DEFAULT NULL,
    `fatal` VARCHAR (45) DEFAULT NULL,
    `sex` VARCHAR (45) DEFAULT NULL)
    
    ENGINE = InnoDB;
    '''

- 4. Llamamos a la función para crear dicha tabla.

In [49]:
bbdd_b = carga.create_insert_table(bbdd_a, attacks)

- 5. Creamos una segunda query iterando por cada columna y fila del dataframe.
     Llamamos de nuevo a la función, esta vez para insertar los datos.

In [50]:
for indice, fila in df_at.iterrows():
    
    query_x = f"""
            INSERT INTO attacks (year, type, country, activity, age, species, month, fatal, sex)
            VALUES ( "{fila["year"]}", "{fila['type']}", "{fila['country']}", "{fila['activity']}", "{fila['age']}", 
            "{fila['species']}", "{fila['month']}", "{fila['type']}", "{fila['sex']}");
            """
# La función debe ser llamada dentro del for, ya que si no, no itera por todos los valores y sólo nos inserta la primera línea.
    bbdd_c = carga.create_insert_table(bbdd_a, query_x)